# Create KB

In [1]:
%cd ..

/home/codeholder/code/python-playground/app_noisemon


In [29]:
import json
import os
from pathlib import Path
from tqdm import tqdm
import spacy
from spacy.kb import KnowledgeBase
from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
msg = Printer()
from scripts.convert_labelstudio_to_spacy import LabelStudioToSpacyConverter

In [38]:
input_path = Path("./data/05-labeled")
output_folder = Path("./corpus/")
kb_path = Path("./corpus/noisemon_kb")

In [39]:
files = list(input_path.glob("*.json"))
if files:
    input_path = max(files, key=os.path.getctime)
else:
    msg.fail(f"Directory {input_path} is empty")

In [40]:
data = json.loads(input_path.read_text())

In [41]:
data[0]

{'id': 334657,
 'annotations': [{'id': 87,
   'completed_by': {'id': 1,
    'email': 'maksim_ermakov@protonmail.com',
    'first_name': '',
    'last_name': ''},
   'result': [{'value': {'start': 6,
      'end': 22,
      'text': 'Открытие Капитал',
      'labels': ['ORG']},
     'id': '74520158391160',
     'from_name': 'ner',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 6, 'end': 22, 'text': ['Q108398998']},
     'id': '74520158391160',
     'from_name': 'entity',
     'to_name': 'text',
     'type': 'textarea'},
    {'value': {'start': 101,
      'end': 109,
      'text': 'Роснефти',
      'labels': ['ORG']},
     'id': '18591563568703',
     'from_name': 'ner',
     'to_name': 'text',
     'type': 'labels'},
    {'value': {'start': 101, 'end': 109, 'text': ['Q1141123']},
     'id': '18591563568703',
     'from_name': 'entity',
     'to_name': 'text',
     'type': 'textarea'},
    {'value': {'start': 198,
      'end': 206,
      'text': 'Роснефти',
      '

In [42]:
nlp_model = "ru_core_news_sm"
nlp_model_vector_size = 96
nlp = spacy.load(nlp_model, exclude="parser, tagger")
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=nlp_model_vector_size)

In [43]:
vocab = Vocab()

# 1. Create NER Spacy Dataset
output = []
converter = LabelStudioToSpacyConverter()
converter.ls_label_map = {
    "ORG": "ORG"
}

for labelstudio_line in data:
    doc = converter.create_spacy_doc(labelstudio_line)
    doc = converter.assign_entities(doc, labelstudio_line)
    output.append(doc)

In [44]:
from collections import defaultdict

In [45]:
# 1. SEEDING KB ENTITES
for i in tqdm(range(len(data))):
    id_to_qid_name_pair = defaultdict(dict)
    for chunk in data[i]["annotations"][0]["result"]:
        if chunk["from_name"] == "ner":
            id_to_qid_name_pair[chunk["id"]]["text"] = chunk["value"]["text"]
        if chunk["from_name"] == "entity":
            id_to_qid_name_pair[chunk["id"]]["qid"] = chunk["value"]["text"][0]
    for _, pair in id_to_qid_name_pair.items():
        if "qid" not in pair:
            msg.fail("QID is missing!", data[i]["data"]["text"])
            continue
        vector = nlp(data[i]["data"]["text"]).vector
        kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)

  0%|                                                                                                                                                | 0/79 [00:00<?, ?it/s]/tmp/ipykernel_11411/2273782613.py:14: UserWarning: [W018] Entity 'Q1141123' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_11411/2273782613.py:14: UserWarning: [W018] Entity 'Q940518' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_11411/2273782613.py:14: UserWarning: [W018] Entity 'Q1355823' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
  3%|███▍                                                                                                                                    | 2/79 [00:00<00:04, 17.93it/s]/tmp/ipykernel_

✘ QID is missing!
RUAL отчет Чистая прибыль компании «Русал» за январь—сентябрь 2017 года по ​по
международным стандартам финансовой отчетности (МСФО) ​составила ​$782 млн,
превысив на ​46,4% аналогичный показатель прошлого года. Об этом говорится в
отчете о финансовых результатах, опубликованном на сайте компании. При этом
выручка компании за третий квартал увеличилась на 19,4% и составила $2,46 млрд,
а за девять месяцев текущего года — на 21,3%, до $7,224 млрд. Росту финансовых
показателей способствовало увеличение цены алюминия на Лондонской бирже металлов
(LME), говорится в отчете.


/tmp/ipykernel_11411/2273782613.py:14: UserWarning: [W018] Entity 'Q1642605' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
/tmp/ipykernel_11411/2273782613.py:14: UserWarning: [W018] Entity 'Q1720713' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
 77%|████████████████████████████████████████████████████████████████████████████████████████████████████████▏                              | 61/79 [00:02<00:00, 22.44it/s]/tmp/ipykernel_11411/2273782613.py:14: UserWarning: [W018] Entity 'Q1781702' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=pair["qid"], entity_vector=vector, freq=265)
 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                         | 64/79 [00:02<00:00, 22.21it/s]/tmp/ipykernel

In [46]:
# 2. SEEDING KB ALIASES
for i in tqdm(range(len(data))):
    id_to_qid_name_pair = defaultdict(dict)
    for chunk in data[i]["annotations"][0]["result"]:
        if chunk["from_name"] == "ner":
            id_to_qid_name_pair[chunk["id"]]["text"] = chunk["value"]["text"]
        if chunk["from_name"] == "entity":
            id_to_qid_name_pair[chunk["id"]]["qid"] = chunk["value"]["text"][0]
    for _, pair in id_to_qid_name_pair.items():
        if "qid" not in pair:
            msg.fail("QID is missing!", data[i]["data"]["text"])
            continue
        vector = nlp(data[i]["data"]["text"]).vector
        kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])

  0%|                                                                                                                                                | 0/79 [00:00<?, ?it/s]/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'Роснефти' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'Магнита' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'FTSE' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
  3%|███▍                                                                                                                                    | 2/79 [00:00<00:04, 17.68it/s]/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'РуссНефти' already 

✘ QID is missing!
RUAL отчет Чистая прибыль компании «Русал» за январь—сентябрь 2017 года по ​по
международным стандартам финансовой отчетности (МСФО) ​составила ​$782 млн,
превысив на ​46,4% аналогичный показатель прошлого года. Об этом говорится в
отчете о финансовых результатах, опубликованном на сайте компании. При этом
выручка компании за третий квартал увеличилась на 19,4% и составила $2,46 млрд,
а за девять месяцев текущего года — на 21,3%, до $7,224 млрд. Росту финансовых
показателей способствовало увеличение цены алюминия на Лондонской бирже металлов
(LME), говорится в отчете.


/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'МегаФона' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
 77%|████████████████████████████████████████████████████████████████████████████████████████████████████████▏                              | 61/79 [00:02<00:00, 23.26it/s]/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'Алроса' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'АФК «Система»' already exists in the Knowledge Base.
  kb.add_alias(alias=pair["text"], entities=[pair["qid"]], probabilities=[1])
 82%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████                        | 65/79 [00:02<00:00, 23.10it/s]/tmp/ipykernel_11411/2171502097.py:14: UserWarning: [W017] Alias 'Интерфаксом

In [47]:
id_to_qid_name_pair

defaultdict(dict,
            {'48983422152598': {'qid': 'Q1023876', 'text': 'CME Group'},
             'AtuzZC1kQa': {'text': 'биткоин', 'qid': 'Q131723'}})

In [48]:
kb

In [49]:
kb.to_disk(kb_path)

# nlp.to_disk(nlp_dir)